In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [3]:
#80% train test split
np.random.seed(42)
n_samples = 1000
X = np.random.uniform(-1, 1, (n_samples, 1))
y = np.sin(3 * np.pi * X) + 0.1 * np.random.randn(n_samples, 1)
X_train, y_train = X[:800], y[:800]
X_test, y_test = X[800:], y[800:]


In [4]:
X_train = torch.from_numpy(X_train).float().to(device)
y_train = torch.from_numpy(y_train).float().to(device)
X_test = torch.from_numpy(X_test).float().to(device)
y_test = torch.from_numpy(y_test).float().to(device)


In [5]:
class Net(torch.nn.Module):
    def __init__(self, hidden_units):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(1, hidden_units)
        self.fc2 = torch.nn.Linear(hidden_units, hidden_units)
        self.fc3 = torch.nn.Linear(hidden_units, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [6]:
def train_and_evaluate_model(model, hidden_units, optimizer, criterion):
    train_losses = []
    test_losses = []
    for epoch in range(1000):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

        with torch.no_grad():
            outputs = model(X_test)
            test_loss = criterion(outputs, y_test)
            test_losses.append(test_loss.item())

    return test_losses[-1]

In [ ]:
hidden_units_list = [i for i in range(1, 8192, 10)]
test_mses = []
for hidden_units in hidden_units_list:
    model = Net(hidden_units)
    model = model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    criterion = torch.nn.MSELoss()
    test_mse = train_and_evaluate_model(model, hidden_units, optimizer, criterion)
    test_mses.append(test_mse)

plt.plot(hidden_units_list, test_mses, '-o')
plt.xscale('log')
plt.xlabel('Number of Hidden Units')
plt.ylabel('Test MSE')
plt.show()
